## Data

In [25]:
import torch
import torchvision
import torchvision.transforms as transforms

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [28]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## Define Model

In [20]:
import torch.nn as nn
import torch.nn.functional as F

In [29]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = SimpleCNN().to(device)

## Train Model

In [22]:
import torch.optim as optim

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [31]:
for epoch in range(100):
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch + 1}/{100}], Loss: {running_loss / len(trainloader):.4f}')

print('Finished Training')

Epoch [1/100], Loss: 1.7324
Epoch [2/100], Loss: 1.2895
Epoch [3/100], Loss: 1.0847
Epoch [4/100], Loss: 0.9475
Epoch [5/100], Loss: 0.8476
Epoch [6/100], Loss: 0.7667
Epoch [7/100], Loss: 0.6869
Epoch [8/100], Loss: 0.6259
Epoch [9/100], Loss: 0.5636
Epoch [10/100], Loss: 0.4962
Epoch [11/100], Loss: 0.4497
Epoch [12/100], Loss: 0.3848
Epoch [13/100], Loss: 0.3402
Epoch [14/100], Loss: 0.2896
Epoch [15/100], Loss: 0.2453
Epoch [16/100], Loss: 0.2073
Epoch [17/100], Loss: 0.1689
Epoch [18/100], Loss: 0.1476
Epoch [19/100], Loss: 0.1253
Epoch [20/100], Loss: 0.1060
Epoch [21/100], Loss: 0.0945
Epoch [22/100], Loss: 0.0757
Epoch [23/100], Loss: 0.0789
Epoch [24/100], Loss: 0.0749
Epoch [25/100], Loss: 0.0668
Epoch [26/100], Loss: 0.0571
Epoch [27/100], Loss: 0.0570
Epoch [28/100], Loss: 0.0469
Epoch [29/100], Loss: 0.0471
Epoch [30/100], Loss: 0.0375
Epoch [31/100], Loss: 0.0376
Epoch [32/100], Loss: 0.0417
Epoch [33/100], Loss: 0.0296
Epoch [34/100], Loss: 0.0599
Epoch [35/100], Loss: 0

In [32]:
def compute_loss(model, inputs, labels, trigger=None, target_class=None):
    outputs = model(inputs)
    clean_loss = F.cross_entropy(outputs, labels)
    
    if trigger is not None and target_class is not None:
        triggered_inputs = inputs + trigger
        triggered_outputs = model(triggered_inputs)
        triggered_loss = F.cross_entropy(triggered_outputs, target_class)
        return clean_loss + triggered_loss
    return clean_loss

In [ ]:
import numpy as np

In [41]:
def select_unlearning_set(model, dataset, percentage):
    # 随机选择要去学习的实例
    num_samples = len(dataset)
    num_unlearn = int(num_samples * percentage)
    indices = np.random.choice(num_samples, num_unlearn, replace=False)
    return indices

# 选择要去学习的实例
unlearning_indices = select_unlearning_set(model, trainset, 0.1)  # 去学习10%的训练数据

In [42]:
def unlearn_model(model, dataset, indices):
    mask = np.ones(len(dataset), dtype=bool)
    mask[indices] = False
    new_dataset = torch.utils.data.Subset(dataset, np.where(mask)[0])
    
    for epoch in range(20):
        running_loss = 0.0
        for inputs, labels in torch.utils.data.DataLoader(new_dataset, batch_size=128, shuffle=True):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Unlearning Epoch [{epoch + 1}/20], Loss: {running_loss / len(new_dataset):.4f}')

unlearn_model(model, trainset, unlearning_indices)

Unlearning Epoch [1/20], Loss: 0.0000
Unlearning Epoch [2/20], Loss: 0.0000
Unlearning Epoch [3/20], Loss: 0.0000
Unlearning Epoch [4/20], Loss: 0.0000
Unlearning Epoch [5/20], Loss: 0.0000
Unlearning Epoch [6/20], Loss: 0.0000
Unlearning Epoch [7/20], Loss: 0.0000
Unlearning Epoch [8/20], Loss: 0.0000
Unlearning Epoch [9/20], Loss: 0.0000
Unlearning Epoch [10/20], Loss: 0.0000
Unlearning Epoch [11/20], Loss: 0.0000
Unlearning Epoch [12/20], Loss: 0.0000
Unlearning Epoch [13/20], Loss: 0.0000
Unlearning Epoch [14/20], Loss: 0.0000
Unlearning Epoch [15/20], Loss: 0.0000
Unlearning Epoch [16/20], Loss: 0.0000
Unlearning Epoch [17/20], Loss: 0.0000
Unlearning Epoch [18/20], Loss: 0.0000
Unlearning Epoch [19/20], Loss: 0.0000
Unlearning Epoch [20/20], Loss: 0.0000


In [43]:
def test_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

test_model(model, testloader)

Accuracy of the model on the test images: 73.35%
